In [24]:
import time
import hopsworks
import os
from dotenv import load_dotenv
import joblib

# from datetime import datetime, timedelta
import pandas as pd

import hopsworks
import matplotlib.pyplot as plt

from utils import *
import utils


In [25]:
today = int(time.time())
seconds_in_one_day = 24 * 60 * 60
today

1736000416

In [26]:
load_dotenv()
#os.environ["HOPSWORKS_API_KEY"] = os.getenv("HOPSWORKS_API_KEY")
project = hopsworks.login()
#secrets = secrets_api(project.name)
# This line will fail if you have not registered the CRYPTO_API_KEY as a secret in Hopsworks
#CRYPTO_API_KEY = secrets.get_secret("CRYPTO_API_KEY").value

#CRYPTO_API_KEY = os.getenv("CRYPTO_API_KEY")

2025-01-04 16:20:16,438 INFO: Closing external client and cleaning up certificates.
Connection closed.
2025-01-04 16:20:16,444 INFO: Initializing external client
2025-01-04 16:20:16,445 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-01-04 16:21:04,277 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1160346


In [27]:
fs = project.get_feature_store() 
starting_timestamp = float(today - 40 * seconds_in_one_day)
starting_timestamp


1732544416.0

In [28]:
today, starting_timestamp

(1736000416, 1732544416.0)

In [29]:
solana_feature_group = fs.get_feature_group(
    name="solana",  
    version=1
)

solana_batch_data = solana_feature_group.filter(solana_feature_group.timestamp >= starting_timestamp).read().sort_values(by='timestamp', inplace=False).reset_index(drop=True)
solana_batch_data.head()

Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (1.05s) 


,timestamp,open,high,low,close,volume,volume_buy,volume_sell
0,1732579200,233.96,239.68,221.82,230.68,445887.775346,201503.607457,244384.167889
1,1732665600,230.68,245.00,227.66,242.26,265621.873707,161330.187523,104291.686184
2,1732752000,242.26,245.46,233.00,237.59,142466.637324,73142.398338,69324.238987
3,1732838400,237.59,247.04,236.58,243.47,301558.576846,121169.466633,180389.110213
4,1732924800,243.47,245.34,237.56,237.75,173213.101124,81975.271321,91237.829804


In [30]:
bitcoin_feature_group = fs.get_feature_group(
    name="bitcoin",  
    version=1
)
bitcoin_batch_data = bitcoin_feature_group.filter(bitcoin_feature_group.timestamp >= starting_timestamp).read().sort_values(by='timestamp', inplace=False).reset_index(drop=True)
bitcoin_batch_data = bitcoin_batch_data[["timestamp","open"]]
bitcoin_batch_data.head()

Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (0.63s) 


,timestamp,open
0,1732579200,93000.0
1,1732665600,91914.5
2,1732752000,95952.9
3,1732838400,95637.8
4,1732924800,97478.7


In [31]:
fng_feature_group = fs.get_feature_group(
    'f_n_g_index',
    version=1
)
fng_batch_data = fng_feature_group.filter(fng_feature_group.timestamp >= starting_timestamp).read().sort_values(by='timestamp', inplace=False).reset_index(drop=True)
fng_batch_data['value_classification'] = fng_batch_data['value_classification'].map(fng_labels_to_numbers_map)
fng_batch_data.head()

Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (0.59s) 


,value,value_classification,timestamp
0,79,4,1.732579e+09
1,75,3,1.732666e+09
2,77,4,1.732752e+09
3,78,4,1.732838e+09
4,84,4,1.732925e+09


In [32]:
from tensorflow.keras.models import load_model

mr = project.get_model_registry()

retrieved_model = mr.get_model(
    name="solana_price_daily_model_10d",
    version=3,
)
saved_model_dir = retrieved_model.download()
print(saved_model_dir)
model = load_model(saved_model_dir+'/best_model.keras')
model


/var/folders/rh/xp6gtwpn19s_8lqdxq8sdl0r0000gn/T/d7ee57ea-cf89-4ced-bd8f-fa28322721d6/solana_price_daily_model_10d/3


<Sequential name=sequential, built=True>

In [33]:
print(solana_batch_data['timestamp'].dtype)
print(bitcoin_batch_data['timestamp'].dtype)
print(fng_batch_data['timestamp'].dtype)

print(solana_batch_data['timestamp'].head())
print(bitcoin_batch_data['timestamp'].head())
print(fng_batch_data['timestamp'].head())


int64
int64
float64
0    1732579200
1    1732665600
2    1732752000
3    1732838400
4    1732924800
Name: timestamp, dtype: int64
0    1732579200
1    1732665600
2    1732752000
3    1732838400
4    1732924800
Name: timestamp, dtype: int64
0    1.732579e+09
1    1.732666e+09
2    1.732752e+09
3    1.732838e+09
4    1.732925e+09
Name: timestamp, dtype: float64


In [34]:
all_data = pd.merge(solana_batch_data, bitcoin_batch_data, how='inner', on="timestamp",suffixes=('_solana', '_bitcoin'))  
all_data = pd.merge(all_data, fng_batch_data, how='inner', on="timestamp")

if "timestamp" in all_data.columns:
    all_data = all_data.drop(columns=['timestamp'])

target_column = 'open_solana'
all_data = utils.move_target_at_the_end(all_data, target_column)
all_data_values = all_data.values[-30:]

all_data.head()

,high,low,close,volume,volume_buy,volume_sell,open_bitcoin,value,value_classification,open_solana
0,239.68,221.82,230.68,445887.775346,201503.607457,244384.167889,93000.0,79,4,233.96
1,245.00,227.66,242.26,265621.873707,161330.187523,104291.686184,91914.5,75,3,230.68
2,245.46,233.00,237.59,142466.637324,73142.398338,69324.238987,95952.9,77,4,242.26
3,247.04,236.58,243.47,301558.576846,121169.466633,180389.110213,95637.8,78,4,237.59
4,245.34,237.56,237.75,173213.101124,81975.271321,91237.829804,97478.7,84,4,243.47


In [35]:
scaler_labels = joblib.load('scaler_labels.pkl')
scaler_features = joblib.load('scaler_features.pkl')

all_data_values = scaler_features.transform(all_data_values)
X = all_data_values.reshape(1, 30, 10)


In [36]:
all_data.columns

Index(['high', 'low', 'close', 'volume', 'volume_buy', 'volume_sell',
       'open_bitcoin', 'value', 'value_classification', 'open_solana'],
      dtype='object')

In [37]:
predictions = model.predict(X)[0]

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 112ms/step


In [38]:
predictions

array([0.4165244 , 0.5076676 , 0.42782706, 0.30283636, 0.32652402,
       0.52912307, 0.52718985, 0.7732368 , 0.46171755, 0.46803877],
      dtype=float32)

In [39]:
range(len(predictions))

range(0, 10)

In [40]:

timestamps = [today + (i+1) * seconds_in_one_day for i in range(len(predictions))]

In [41]:
monitor_fg = fs.get_or_create_feature_group(
    name='solana_predictions',
    description='Solana_predictions',
    version=1,
    primary_key=['date']
)

In [42]:
result = pd.DataFrame(columns=['timestamp', "open_solana"])
result["timestamp"]=timestamps
result["open_solana"]=predictions
#result= result.sort_values(by=['timestamp'])

In [43]:
result

,timestamp,open_solana
0,1736086816,0.416524
1,1736173216,0.507668
2,1736259616,0.427827
3,1736346016,0.302836
4,1736432416,0.326524
5,1736518816,0.529123
6,1736605216,0.527190
7,1736691616,0.773237
8,1736778016,0.461718
9,1736864416,0.468039


In [44]:
result['date'] = pd.to_datetime(result['timestamp'], unit='s')
result['date'] = result['date'].dt.normalize()
result.drop(columns=["timestamp"], inplace=True)

result

,open_solana,date
0,0.416524,2025-01-05
1,0.507668,2025-01-06
2,0.427827,2025-01-07
3,0.302836,2025-01-08
4,0.326524,2025-01-09
5,0.529123,2025-01-10
6,0.527190,2025-01-11
7,0.773237,2025-01-12
8,0.461718,2025-01-13
9,0.468039,2025-01-14


In [45]:
#this is plane and we may have duplicates
#monitor_fg.insert(result)

In [46]:
#this insert prevents duplicates
monitor_fg.insert(
    result,
    #overwrite=False,  # Do not overwrite all data
    operation="upsert",  # Use upsert to handle duplicates based on the primary key
    #storage=None  # Write to both online and offline storage
)

Uploading Dataframe: 100.00% |██████████| Rows 10/10 | Elapsed Time: 00:02 | Remaining Time: 00:00


Launching job: solana_predictions_1_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai:443/p/1160346/jobs/named/solana_predictions_1_offline_fg_materialization/executions


(Job('solana_predictions_1_offline_fg_materialization', 'SPARK'), None)